### *IT3052E - Fundamentals of Optimization*
# **Mini Project 18 - Nurse Scheduling Problem**
#### **Techniques used**:
* Backtracking,
* Constraint Programming,
* Linear Programming,
* Local Search, and
* Meta-heuristics (Genetic Algorithm).

#### * ***Import pandas for printing solution***

In [34]:
import pandas as pd
import time

## 3. Hill-climbing search with PyCBLS

### 3.1. Import modules and libraries

In [35]:
from PyCBLS.VarIntLS import VarIntLS
from PyCBLS.LocalSearchManager import LocalSearchManager
from PyCBLS.NotEqual import NotEqual
from PyCBLS.ConstraintSystem import ConstraintSystem
from PyCBLS.HillClimbingSearch import HillClimbingSearch
from PyCBLS.ConditionalSumWithBound import ConditionalSumWithBound
import random as rd

### 3.2. Read data from files

In [36]:
with open('SampleData/testCase1/0.txt') as file:
  N, D, a, b = [int(q) for q in file.readline().split()]
  dayoff = [[0 for d in range(D)] for n in range(N)]
  for n in range(N):
    for d in [int(h) for h in file.readline().split()]:
      if d != -1:
            dayoff[n][d-1] = 1
            
starttime = time.time()

### 3.3. Generate local search and define decision variable

In [37]:
mgr = LocalSearchManager()

LocalSearchManagerconstructor


In [38]:
assign = [[VarIntLS(mgr,0,4,rd.randint(1, 4),f'assign_{n}_{d}') for n in range(N)] for d in range(D)]

mor = VarIntLS(mgr,1,1,1,'one')
aft = VarIntLS(mgr,2,2,2,'one')
eve = VarIntLS(mgr,3,3,3,'one')
nig = VarIntLS(mgr,4,4,4,'one')
zero = VarIntLS(mgr,0,0,0,'zero')

### 3.4. Generate constraints

In [39]:
constraints = []

#### 3.4.1. A nurse can be assigned to only one shift per day.

This is constrained by creating decision variable

#### 3.4.2. Each shift has min $a$ nurses and max $b$ nurses.

In [40]:
count = [1 for i in range(N)]
for d in range(D):
    c = ConditionalSumWithBound(assign[d],count,1,a,b,'CountMor')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,2,a,b,'CountAft')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,3,a,b,'CountEve')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,4,a,b,'CountNig')
    constraints.append(c)

#### 3.4.3. Set the given dayoff and dayoff after a night shift

In [41]:
for n in range(N):
    for d in range(D-1):
        if dayoff[n][d] == 1 or assign[d-1][n] == nig:
            c = NotEqual(assign[d][n],mor,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],aft,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],eve,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],nig,'NotEqual')
            constraints.append(c)
        else:
            c = NotEqual(assign[d][n],zero,'NotEqual')
            constraints.append(c)
            
    # handle the last day
    if dayoff[n][D-1] == 1:
        c = NotEqual(assign[D-1][n],mor,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],aft,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],eve,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],nig,'NotEqual')
        constraints.append(c)
    else:
        c = NotEqual(assign[D-1][n],zero,'NotEqual')
        constraints.append(c)

### 3.5. Start local search

In [42]:
C = ConstraintSystem(constraints)
mgr.close()

print('Init, C = ',C.violations())
searcher = HillClimbingSearch(C)
searcher.search(5000)

ConstraintSystem::constructor
LocalSearchManager::close
topoSortInvariants finished
Init, C =  16
0 : assign x[ 37 ] =  0  violations =  15
1 : assign x[ 2 ] =  0  violations =  14
2 : assign x[ 3 ] =  0  violations =  13
3 : assign x[ 18 ] =  0  violations =  12
4 : assign x[ 30 ] =  0  violations =  11
5 : assign x[ 13 ] =  0  violations =  10
6 : assign x[ 58 ] =  0  violations =  9
7 : assign x[ 1 ] =  0  violations =  8
8 : assign x[ 10 ] =  0  violations =  7
9 : assign x[ 46 ] =  0  violations =  6
10 : assign x[ 36 ] =  0  violations =  5
11 : assign x[ 44 ] =  0  violations =  4
12 : assign x[ 32 ] =  0  violations =  3
13 : assign x[ 45 ] =  0  violations =  2
14 : assign x[ 60 ] =  0  violations =  1
15 : assign x[ 38 ] =  0  violations =  0
Violations =  0


### 3.6. Print the schedule

In [43]:
res = [[0 for n in range(N)] for d in range(D)]
for d in range(D):
    for n in range(N):
        res[d][n] = assign[d][n].getValue()
df = pd.DataFrame(res, index = [d+1 for d in range(D)], columns = [n+1 for n in range(N)])
df.index.name = 'Nurse'
df.columns.name = 'Day'
display(df)
print('Running time:',time.time() - starttime)

Day,1,2,3,4,5,6,7,8,9,10
Nurse,,,,,,,,,,
1,4,3,4,0,1,3,0,1,2,3
2,4,3,0,3,0,0,2,4,0,2
3,1,2,1,0,2,0,2,4,1,2
4,1,0,1,4,0,3,0,2,4,3
5,0,1,1,4,3,0,4,4,0,2
6,4,0,4,4,0,1,3,3,2,3


Running time: 0.08660364151000977


### 3.7. Optimal solution

In [44]:
maxNightShift = -1
for n in range(N):
    tmp = 0
    for d in range(D):
        if res[d][n] == 4:
            tmp += 1
    if tmp > maxNightShift:
        maxNightShift = tmp
        
print('Optimal solution - Max night shift assigned to a nurse:', maxNightShift)

Optimal solution - Max night shift assigned to a nurse: 3
